In [1]:
from numpy.random import choice
from pandas import read_csv, DataFrame
from sklearn.cluster import KMeans
import os, sys
# sys.path.insert(0, 'C:/Users/jjung/Documents/GitHub//bkmark_organizer/test_parser_stemmer/prototype/TxtClus/')
sys.path.insert(0, '/home/jz/proj/News-Spam-Detect/TxtClus')
from nlp.termWeighting import doc_term_matrix
from EstimateK.seqFit import sensitiv

In [2]:
class Clusterings(object):
    '''Define a class that encapsulates textual processing tools.'''
    def __init__(self, param_dict):
        self.__param_dict = param_dict        
    
    def get_file(self):
        '''read csv input into a pandas data frame'''
        return read_csv(self.__param_dict['file_loc'], encoding = 'latin1')

    def term_weight_matr(self, snippetsArr):
        '''compute a document-term matrix based on a collection of text documents'''
        return doc_term_matrix(snippetsArr, self.__param_dict)
    
    def resample(self, df, NUM_BOOTSTRAPS = 3):
        bootstraps = [None] * NUM_BOOTSTRAPS
        for bootI in range(NUM_BOOTSTRAPS):  
            bootstraps[bootI] = df.sample(frac=1/NUM_BOOTSTRAPS, replace=True)
        return bootstraps

In [3]:
if __name__ == "__main__":
    # Pass in settings to instantiate a Clusterings object called vecSpaceMod1:
    vecSpaceMod = Clusterings({'file_loc': sys.path[0] + '/Input/newsSample.csv',
                               'tf_dampen': True,
                               'common_word_pct': 1,
                               'rare_word_pct': 1,
                               'dim_redu': False})
    df = vecSpaceMod.get_file() # Load csv file into data frame.

In [4]:
    # Take 3 bootstrap sub-samples for faster, bagged kmeans fits:
    bstraps = vecSpaceMod.resample(df)
    # Compute the Term Frequency Inverse Document Frequency matrix based on news headlines:    
    design_matrices = [vecSpaceMod.term_weight_matr(bstrap.TITLE) for bstrap in bstraps]

In [8]:
    [sensitiv(matrix) for matrix in design_matrices] 

,2,3,4,5,6,7,8,9,10,11,...,21,22,23,24,25,26,27,28,29,30
0,0.095004,0.158038,0.171808,0.216971,0.206889,0.238394,0.277985,0.292249,0.281520,0.342076,...,0.317505,0.315022,0.317563,0.317039,0.305056,0.296807,0.258065,0.258065,0.258065,0.258065
1,3.089662,2.704130,2.520869,2.507684,2.357642,2.329167,2.404771,2.335299,2.284718,2.414004,...,4.023399,4.578704,5.264197,6.363144,9.017937,17.258041,1.000000,1.000000,1.000000,1.000000


In [ ]:
    df = DataFrame({'predictedCluster': KMeans(17).fit(X1).labels_, 
                    'document': term_weight_obj['samp']}).sort_values(by='predictedCluster')
    
    print(df)